**Seeding for reproducibility**

In [1]:
# Set seeds for reproducibility
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)

**Importing the dependencies**

In [2]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

**Data Curation**

Upload the kaggle.json file

In [ ]:
!pip install kaggle

In [4]:
kaggle_credentails = json.load(open("kaggle.json"))

In [5]:
# setup Kaggle API key as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_credentails["username"]
os.environ['KAGGLE_KEY'] = kaggle_credentails["key"]

In [ ]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

In [7]:
!ls

kaggle.json  plantvillage-dataset.zip  sample_data


In [8]:
# Unzip the downloaded dataset
with ZipFile("plantvillage-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [10]:
print(os.listdir("plantvillage dataset"))


print(len(os.listdir("plantvillage dataset/segmented")))
print(os.listdir("plantvillage dataset/segmented")[:5])

print(len(os.listdir("plantvillage dataset/color")))
print(os.listdir("plantvillage dataset/color")[:5])

print(len(os.listdir("plantvillage dataset/grayscale")))
print(os.listdir("plantvillage dataset/grayscale")[:5])

['grayscale', 'segmented', 'color']
38
['Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Bacterial_spot', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy']
38
['Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Bacterial_spot', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy']
38
['Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Bacterial_spot', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy']


**Number of Classes = 38**

In [11]:
print(len(os.listdir("plantvillage dataset/color/Grape___healthy")))
print(os.listdir("plantvillage dataset/color/Grape___healthy")[:5])

423
['05cff9d7-0f63-4b6e-9aa3-199cf9ffa64c___Mt.N.V_HL 9111.JPG', '0ce12a10-c6ff-494e-a927-5ddc809c707a___Mt.N.V_HL 8945.JPG', 'c839e134-6692-4065-8e12-7ea01adcc794___Mt.N.V_HL 9014.JPG', '452823c1-22d2-4bce-a4cc-8ad014421261___Mt.N.V_HL 6199.JPG', 'c48a4277-3bd4-45c8-be06-8bdc4404f252___Mt.N.V_HL 6137.JPG']


**Data Preprocessing**

In [12]:
# Dataset Path
base_dir = 'plantvillage dataset/color'

In [ ]:
image_path = '/content/plantvillage dataset/color/Apple___Cedar_apple_rust/025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655.JPG'

# Read the image
img = mpimg.imread(image_path)

print(img.shape)
# Display the image
plt.imshow(img)
plt.axis('off')  # Turn off axis numbers
plt.show()

In [14]:
image_path = '/content/plantvillage dataset/color/Apple___Cedar_apple_rust/025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655.JPG'

# Read the image
img = mpimg.imread(image_path)

print(img)

[[[179 175 176]
  [181 177 178]
  [184 180 181]
  ...
  [115 112 105]
  [108 105  98]
  [101  98  91]]

 [[176 172 173]
  [177 173 174]
  [178 174 175]
  ...
  [113 110 103]
  [111 108 101]
  [109 106  99]]

 [[180 176 177]
  [180 176 177]
  [180 176 177]
  ...
  [108 105  98]
  [111 108 101]
  [114 111 104]]

 ...

 [[137 128 119]
  [131 122 113]
  [125 116 107]
  ...
  [ 74  65  48]
  [ 74  65  48]
  [ 73  64  47]]

 [[136 127 118]
  [132 123 114]
  [128 119 110]
  ...
  [ 77  69  50]
  [ 75  67  48]
  [ 75  67  48]]

 [[133 124 115]
  [133 124 115]
  [132 123 114]
  ...
  [ 81  73  54]
  [ 80  72  53]
  [ 79  71  52]]]


In [15]:
# Image Parameters
img_size = 224
batch_size = 32

**Train Test Split**

In [16]:
# Image Data Generators
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)

In [17]:
# Train Generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 43456 images belonging to 38 classes.


In [18]:
# Validation Generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 10849 images belonging to 38 classes.


**Convolutional Neural Network**

In [20]:
# Model Definition
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

In [21]:
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 256)               4

In [22]:
# Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Model training**

In [23]:
# Training the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Number of steps per epoch
    epochs=5,  # Number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size  # Validation steps
)

Epoch 1/5
1358/1358 [==============================] - 108s 76ms/step - loss: 0.9791 - accuracy: 0.7328 - val_loss: 0.4846 - val_accuracy: 0.8465
Epoch 2/5
1358/1358 [==============================] - 104s 77ms/step - loss: 0.2812 - accuracy: 0.9110 - val_loss: 0.4477 - val_accuracy: 0.8655
Epoch 3/5
1358/1358 [==============================] - 106s 78ms/step - loss: 0.1362 - accuracy: 0.9553 - val_loss: 0.4321 - val_accuracy: 0.8863
Epoch 4/5
1358/1358 [==============================] - 103s 76ms/step - loss: 0.0891 - accuracy: 0.9708 - val_loss: 0.5433 - val_accuracy: 0.8715
Epoch 5/5
1358/1358 [==============================] - 109s 81ms/step - loss: 0.0761 - accuracy: 0.9760 - val_loss: 0.5091 - val_accuracy: 0.8828


**Model Evaluation**

In [24]:
# Model Evaluation
print("Evaluating model...")
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating model...
339/339 [==============================] - 19s 57ms/step - loss: 0.5091 - accuracy: 0.8828
Validation Accuracy: 88.28%


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

**Building a Predictive System**

In [26]:
# Function to Load and Preprocess the Image using Pillow
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    img = Image.open(image_path)
    # Resize the image
    img = img.resize(target_size)
    # Convert the image to a numpy array
    img_array = np.array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Scale the image values to [0, 1]
    img_array = img_array.astype('float32') / 255.
    return img_array

# Function to Predict the Class of an Image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [27]:
# Create a mapping from class indices to class names
class_indices = {v: k for k, v in train_generator.class_indices.items()}

In [28]:
class_indices

{0: 'Apple___Apple_scab',
 1: 'Apple___Black_rot',
 2: 'Apple___Cedar_apple_rust',
 3: 'Apple___healthy',
 4: 'Blueberry___healthy',
 5: 'Cherry_(including_sour)___Powdery_mildew',
 6: 'Cherry_(including_sour)___healthy',
 7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 8: 'Corn_(maize)___Common_rust_',
 9: 'Corn_(maize)___Northern_Leaf_Blight',
 10: 'Corn_(maize)___healthy',
 11: 'Grape___Black_rot',
 12: 'Grape___Esca_(Black_Measles)',
 13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 14: 'Grape___healthy',
 15: 'Orange___Haunglongbing_(Citrus_greening)',
 16: 'Peach___Bacterial_spot',
 17: 'Peach___healthy',
 18: 'Pepper,_bell___Bacterial_spot',
 19: 'Pepper,_bell___healthy',
 20: 'Potato___Early_blight',
 21: 'Potato___Late_blight',
 22: 'Potato___healthy',
 23: 'Raspberry___healthy',
 24: 'Soybean___healthy',
 25: 'Squash___Powdery_mildew',
 26: 'Strawberry___Leaf_scorch',
 27: 'Strawberry___healthy',
 28: 'Tomato___Bacterial_spot',
 29: 'Tomato___Early_blight',
 30: '

In [29]:
# saving the class names as json file
json.dump(class_indices, open('class_indices.json', 'w'))

In [31]:
# Example Usage
image_path = '/content/test_apple_black_rot.JPG'
#image_path = '/content/test_blueberry_healthy.jpg'
#image_path = '/content/test_potato_early_blight.jpg'
predicted_class_name = predict_image_class(model, image_path, class_indices)

# Output the result
print("Predicted Class Name:", predicted_class_name)

1/1 [==============================] - 0s 266ms/step
Predicted Class Name: Apple___Black_rot


**Save the model to Google drive or local**

In [ ]:
model.save('drive/MyDrive/trained_models/plant_disease_prediction_model.h5')

In [32]:
model.save('plant_disease_prediction_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Install Additional Dependencies


In [ ]:
!pip install opencv-python-headless seaborn tqdm -q


Import Robustness Testing Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import json
import os

print("✓ Robustness testing libraries loaded")

Corruption Functions

In [ ]:
class ImageCorruptions:
    """Apply realistic corruptions to test images"""
    
    @staticmethod
    def add_gaussian_noise(img, severity=3):
        """Add Gaussian noise (sensor noise)"""
        noise_levels = [0.04, 0.08, 0.12, 0.16, 0.20]
        std = noise_levels[severity - 1]
        noise = np.random.normal(0, std * 255, img.shape)
        noisy = np.clip(img + noise, 0, 255)
        return noisy.astype(np.uint8)
    
    @staticmethod
    def add_salt_pepper_noise(img, severity=3):
        """Salt & pepper noise (transmission errors)"""
        amounts = [0.01, 0.02, 0.03, 0.05, 0.08]
        amount = amounts[severity - 1]
        noisy = np.copy(img)
        
        # Salt
        num_salt = int(amount * img.size * 0.5)
        coords = [np.random.randint(0, i - 1, num_salt) for i in img.shape[:2]]
        noisy[coords[0], coords[1], :] = 255
        
        # Pepper
        num_pepper = int(amount * img.size * 0.5)
        coords = [np.random.randint(0, i - 1, num_pepper) for i in img.shape[:2]]
        noisy[coords[0], coords[1], :] = 0
        
        return noisy
    
    @staticmethod
    def apply_motion_blur(img, severity=3):
        """Motion blur (camera shake)"""
        kernel_sizes = [3, 5, 7, 9, 11]
        size = kernel_sizes[severity - 1]
        kernel = np.zeros((size, size))
        kernel[int((size-1)/2), :] = np.ones(size)
        kernel = kernel / size
        blurred = cv2.filter2D(img, -1, kernel)
        return blurred
    
    @staticmethod
    def apply_gaussian_blur(img, severity=3):
        """Gaussian blur (out of focus)"""
        kernel_sizes = [3, 5, 7, 9, 11]
        ksize = kernel_sizes[severity - 1]
        blurred = cv2.GaussianBlur(img, (ksize, ksize), 0)
        return blurred
    
    @staticmethod
    def adjust_brightness(img, severity=3):
        """Adjust brightness (too dark/bright)"""
        factors = [0.4, 0.6, 0.8, 1.3, 1.6]
        factor = factors[severity - 1]
        adjusted = np.clip(img * factor, 0, 255)
        return adjusted.astype(np.uint8)
    
    @staticmethod
    def adjust_contrast(img, severity=3):
        """Adjust contrast"""
        factors = [0.5, 0.7, 0.9, 1.3, 1.6]
        factor = factors[severity - 1]
        mean = img.mean()
        adjusted = np.clip((img - mean) * factor + mean, 0, 255)
        return adjusted.astype(np.uint8)
    
    @staticmethod
    def apply_fog(img, severity=3):
        """Simulate fog"""
        fog_levels = [0.3, 0.4, 0.5, 0.6, 0.7]
        fog_strength = fog_levels[severity - 1]
        fog = np.ones_like(img) * 200
        foggy = cv2.addWeighted(img, 1 - fog_strength, fog.astype(np.uint8), fog_strength, 0)
        return foggy
    
    @staticmethod
    def add_shadow(img, severity=3):
        """Add shadow effect"""
        shadow_strengths = [0.3, 0.4, 0.5, 0.6, 0.7]
        strength = shadow_strengths[severity - 1]
        
        h, w = img.shape[:2]
        shadow_mask = np.zeros((h, w), dtype=np.float32)
        
        for i in range(h):
            shadow_mask[i, :int(w * (1 - i/h))] = 1
        
        shadow_mask = cv2.GaussianBlur(shadow_mask, (51, 51), 0)
        
        shadowed = img.copy()
        for c in range(3):
            shadowed[:, :, c] = np.clip(
                img[:, :, c] * (1 - strength * shadow_mask), 0, 255
            )
        
        return shadowed.astype(np.uint8)
    
    @staticmethod
    def add_occlusion(img, severity=3):
        """Add random occlusions"""
        occlusion_ratios = [0.1, 0.15, 0.2, 0.25, 0.3]
        ratio = occlusion_ratios[severity - 1]
        
        h, w = img.shape[:2]
        occluded = img.copy()
        
        num_occlusions = np.random.randint(2, 5)
        for _ in range(num_occlusions):
            x1 = np.random.randint(0, w)
            y1 = np.random.randint(0, h)
            size = int(min(h, w) * ratio)
            x2 = min(x1 + size, w)
            y2 = min(y1 + size, h)
            
            color = np.random.randint(20, 80, 3)
            occluded[y1:y2, x1:x2] = color
        
        return occluded

print("✓ Corruption functions defined")

Visualize Sample Corruptions

In [ ]:
def visualize_corruptions(sample_image_path, class_indices):
    """Visualize all corruption types on a sample image"""
    
    # Load sample image
    from PIL import Image
    img = Image.open(sample_image_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    
    corruptions = [
        (ImageCorruptions.add_gaussian_noise, 'Gaussian Noise'),
        (ImageCorruptions.add_salt_pepper_noise, 'Salt & Pepper'),
        (ImageCorruptions.apply_motion_blur, 'Motion Blur'),
        (ImageCorruptions.apply_gaussian_blur, 'Gaussian Blur'),
        (ImageCorruptions.adjust_brightness, 'Dark'),
        (ImageCorruptions.adjust_contrast, 'Low Contrast'),
        (ImageCorruptions.apply_fog, 'Fog'),
        (ImageCorruptions.add_shadow, 'Shadow'),
        (ImageCorruptions.add_occlusion, 'Occlusion'),
    ]
    
    fig, axes = plt.subplots(3, 4, figsize=(15, 12))
    axes = axes.ravel()
    
    # Original image
    axes[0].imshow(img_array)
    axes[0].set_title('Original', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Corrupted images
    for idx, (corrupt_fn, name) in enumerate(corruptions, 1):
        corrupted = corrupt_fn(img_array, severity=3)
        axes[idx].imshow(corrupted)
        axes[idx].set_title(name, fontsize=11)
        axes[idx].axis('off')
    
    # Hide remaining subplots
    for idx in range(len(corruptions) + 1, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Sample Image Corruptions (Severity=3)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('sample_corruptions.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Sample corruptions visualized")


Robustness Evaluator

In [ ]:
def evaluate_robustness(model, validation_generator, class_indices, 
                        severities=[1, 3, 5], max_batches=50):
    """
    Evaluate model robustness on corruptions
    
    Args:
        model: Trained model
        validation_generator: Keras ImageDataGenerator
        class_indices: Dictionary mapping indices to class names
        severities: List of severity levels to test
        max_batches: Maximum batches to test (for speed)
    """
    
    corruptions = [
        (ImageCorruptions.add_gaussian_noise, 'Gaussian Noise'),
        (ImageCorruptions.add_salt_pepper_noise, 'Salt & Pepper'),
        (ImageCorruptions.apply_motion_blur, 'Motion Blur'),
        (ImageCorruptions.apply_gaussian_blur, 'Gaussian Blur'),
        (ImageCorruptions.adjust_brightness, 'Brightness'),
        (ImageCorruptions.adjust_contrast, 'Contrast'),
        (ImageCorruptions.apply_fog, 'Fog'),
        (ImageCorruptions.add_shadow, 'Shadow'),
        (ImageCorruptions.add_occlusion, 'Occlusion'),
    ]
    
    results = []
    
    print("\n" + "="*70)
    print("ROBUSTNESS EVALUATION")
    print("="*70)
    
    for corruption_fn, corruption_name in corruptions:
        for severity in severities:
            print(f"\nTesting: {corruption_name} (Severity {severity})")
            
            y_true = []
            y_pred_clean = []
            y_pred_corrupt = []
            
            # Reset generator
            validation_generator.reset()
            
            for batch_idx in tqdm(range(min(max_batches, len(validation_generator)))):
                images, labels = next(validation_generator)
                
                # Clean predictions
                preds_clean = model.predict(images, verbose=0)
                y_pred_clean.extend(np.argmax(preds_clean, axis=1))
                
                # Apply corruption
                corrupted_images = []
                for img in images:
                    img_uint8 = (img * 255).astype(np.uint8)
                    corrupted = corruption_fn(img_uint8, severity=severity)
                    corrupted_norm = corrupted.astype(np.float32) / 255.0
                    corrupted_images.append(corrupted_norm)
                
                corrupted_batch = np.array(corrupted_images)
                
                # Corrupted predictions
                preds_corrupt = model.predict(corrupted_batch, verbose=0)
                y_pred_corrupt.extend(np.argmax(preds_corrupt, axis=1))
                
                y_true.extend(np.argmax(labels, axis=1))
            
            # Calculate metrics
            acc_clean = accuracy_score(y_true, y_pred_clean)
            acc_corrupt = accuracy_score(y_true, y_pred_corrupt)
            degradation = acc_clean - acc_corrupt
            
            result = {
                'corruption': corruption_name,
                'severity': severity,
                'clean_accuracy': float(acc_clean),
                'corrupted_accuracy': float(acc_corrupt),
                'accuracy_drop': float(degradation),
                'relative_drop_%': float((degradation / acc_clean) * 100) if acc_clean > 0 else 0
            }
            
            results.append(result)
            
            print(f"  Clean: {acc_clean:.3f} | Corrupt: {acc_corrupt:.3f} | Drop: {degradation:.3f}")
    
    return results

print("✓ Robustness evaluator defined")

Visualize Results

In [ ]:
def visualize_robustness_results(results, model_name='Model'):
    """Create visualizations of robustness results"""
    
    df = pd.DataFrame(results)
    
    # Create output directory
    os.makedirs('robustness_results', exist_ok=True)
    
    # 1. Average accuracy drop by corruption type
    fig, ax = plt.subplots(figsize=(12, 6))
    avg_by_corruption = df.groupby('corruption')['accuracy_drop'].mean().sort_values()
    avg_by_corruption.plot(kind='barh', ax=ax, color='coral')
    ax.set_xlabel('Average Accuracy Drop', fontsize=12)
    ax.set_title(f'{model_name} - Robustness to Corruptions', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig('robustness_results/corruption_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. Accuracy vs Severity
    fig, ax = plt.subplots(figsize=(14, 6))
    for corruption in df['corruption'].unique():
        subset = df[df['corruption'] == corruption]
        ax.plot(subset['severity'], subset['corrupted_accuracy'], 
               marker='o', label=corruption, linewidth=2)
    
    ax.set_xlabel('Severity Level', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title(f'{model_name} - Accuracy vs Corruption Severity', fontsize=14, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('robustness_results/severity_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 3. Heatmap of accuracy drop
    pivot = df.pivot(index='corruption', columns='severity', values='accuracy_drop')
    
    fig, ax = plt.subplots(figsize=(8, 8))
    sns.heatmap(pivot, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax, cbar_kws={'label': 'Accuracy Drop'})
    ax.set_title(f'{model_name} - Accuracy Drop Heatmap', fontsize=14, fontweight='bold')
    ax.set_xlabel('Severity Level', fontsize=12)
    ax.set_ylabel('Corruption Type', fontsize=12)
    plt.tight_layout()
    plt.savefig('robustness_results/heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Visualizations saved to 'robustness_results/' folder")


Save Results

In [ ]:
def save_robustness_results(results, model_name='Model'):
    """Save results to CSV and JSON"""
    
    os.makedirs('robustness_results', exist_ok=True)
    
    df = pd.DataFrame(results)
    
    # Save CSV
    csv_path = f'robustness_results/{model_name}_robustness.csv'
    df.to_csv(csv_path, index=False)
    print(f"✓ CSV saved: {csv_path}")
    
    # Save JSON
    json_path = f'robustness_results/{model_name}_robustness.json'
    with open(json_path, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"✓ JSON saved: {json_path}")
    
    # Print summary
    print("\n" + "="*70)
    print("ROBUSTNESS SUMMARY")
    print("="*70)
    print(f"\nAverage accuracy drop: {df['accuracy_drop'].mean():.3f} ({df['relative_drop_%'].mean():.1f}%)")
    print(f"Maximum accuracy drop: {df['accuracy_drop'].max():.3f}")
    print(f"\nMost vulnerable corruptions:")
    print(df.groupby('corruption')['accuracy_drop'].mean().sort_values(ascending=False).head())
    print(f"\nMost robust to:")
    print(df.groupby('corruption')['accuracy_drop'].mean().sort_values().head())

RUN ROBUSTNESS EVALUATION

In [ ]:
# Visualize sample corruptions first
print("Step 1: Visualizing sample corruptions...")
# Pick any test image from your dataset
sample_image = '/content/plantvillage dataset/color/Apple___Black_rot/00a914bb-b9e6-4774-ad3c-1b0a30f96b5e___JR_FrgE.S 2993.JPG'
visualize_corruptions(sample_image, class_indices)

# Run evaluation
print("\nStep 2: Running robustness evaluation...")
print("This will take 10-20 minutes depending on GPU...")

robustness_results = evaluate_robustness(
    model=model,
    validation_generator=validation_generator,
    class_indices=class_indices,
    severities=[1, 3, 5],  # Test mild, moderate, severe
    max_batches=50  # Adjust based on time/accuracy tradeoff
)

# Visualize results
print("\nStep 3: Creating visualizations...")
visualize_robustness_results(robustness_results, model_name='CNN_Model')

# Save results
print("\nStep 4: Saving results...")
save_robustness_results(robustness_results, model_name='CNN_Model')

print("\n" + "="*70)
print("✓ ROBUSTNESS EVALUATION COMPLETE!")
print("="*70)
print("\nCheck the 'robustness_results' folder for:")
print("  - CSV file with detailed results")
print("  - JSON file for further analysis")
print("  - Visualization plots")

Compare Multiple Models

In [ ]:
def compare_multiple_models(model_paths, model_names, validation_generator, class_indices):
    """Compare robustness across multiple models"""
    
    all_results = {}
    
    for model_path, model_name in zip(model_paths, model_names):
        print(f"\n{'='*70}")
        print(f"Evaluating {model_name}")
        print(f"{'='*70}")
        
        # Load model
        from tensorflow import keras
        model = keras.models.load_model(model_path)
        
        # Evaluate
        results = evaluate_robustness(
            model=model,
            validation_generator=validation_generator,
            class_indices=class_indices,
            severities=[3],  # Just test moderate severity for comparison
            max_batches=50
        )
        
        all_results[model_name] = results
    
    # Comparative visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for model_name, results in all_results.items():
        df = pd.DataFrame(results)
        avg_by_corruption = df.groupby('corruption')['accuracy_drop'].mean()
        ax.plot(avg_by_corruption.index, avg_by_corruption.values, 
               marker='o', label=model_name, linewidth=2)
    
    ax.set_xlabel('Corruption Type', fontsize=12)
    ax.set_ylabel('Average Accuracy Drop', fontsize=12)
    ax.set_title('Model Comparison - Robustness to Corruptions', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('robustness_results/model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Model comparison complete!")
    
    return all_results